In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import *

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier

from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV,KFold
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings 
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("D:\\file ds\Alphabets_data.csv")
df.head()

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [3]:
df.shape

(20000, 17)

In [4]:
df.isnull().sum()

letter    0
xbox      0
ybox      0
width     0
height    0
onpix     0
xbar      0
ybar      0
x2bar     0
y2bar     0
xybar     0
x2ybar    0
xy2bar    0
xedge     0
xedgey    0
yedge     0
yedgex    0
dtype: int64

In [5]:
# Separate features and target variable
x = df.drop('letter', axis=1)  # Features
y = df['letter']  # Target variable

In [6]:
# Normalize the data
scaler=StandardScaler()
df_scaled = scaler.fit_transform(x)
df_scaled

array([[-1.0576983 ,  0.29187713, -1.05327668, ..., -0.21908163,
        -1.4381527 ,  0.12291107],
       [ 0.51038497,  1.5023577 , -1.05327668, ..., -0.21908163,
         0.12008142,  1.35944092],
       [-0.01230945,  1.19973756,  0.43590966, ..., -0.8656262 ,
        -0.26947711,  0.74117599],
       ...,
       [ 1.03307939,  0.59449727,  0.43590966, ...,  2.36709667,
        -0.65903564, -2.35014863],
       [-1.0576983 , -1.22122359, -0.55688123, ...,  0.42746295,
         0.50963994,  0.12291107],
       [-0.01230945,  0.59449727,  0.43590966, ..., -0.8656262 ,
        -0.65903564,  0.12291107]])

In [7]:
# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


# Model Implementation

In [8]:
model = Sequential()
#  hidden layers
model.add(Dense(128, input_dim=df_scaled.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
# Output layer
model.add(Dense(len(y.unique()), activation='softmax'))

In [9]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           2,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 26)                  │           1,690 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,122 (47.35 KB)

 Trainable params: 12,122 (47.35 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Split the dataset into training and test sets
x_train, x_test, y_train, y_test = train_test_split(df_scaled, y_encoded, test_size=0.2, random_state=42)

print(x_train.shape) 
print(y_train.shape)

(16000, 16)
(16000,)


In [11]:
# Verify the shapes are consistent
assert x_train.shape[0] == y_train.shape[0]
assert x_test.shape[0] == y_test.shape[0]

# Train the model
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test,y_test))

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4283 - loss: 2.1293 - val_accuracy: 0.7865 - val_loss: 0.7823
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7955 - loss: 0.6981 - val_accuracy: 0.8512 - val_loss: 0.5575
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8484 - loss: 0.5165 - val_accuracy: 0.8660 - val_loss: 0.4564
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8777 - loss: 0.4189 - val_accuracy: 0.8878 - val_loss: 0.3772
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8962 - loss: 0.3492 - val_accuracy: 0.9062 - val_loss: 0.3194
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9106 - loss: 0.2980 - val_accuracy: 0.9060 - val_loss: 0.3120
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9225 - loss: 0.2639 - val_accuracy: 0.9218 - val_loss: 0.2628
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9338 - loss: 0.2269 - val_accuracy: 0.

In [12]:
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(128, input_dim=x_scaled.shape[1], activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(len(y.unique()), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=0)
param_grid = {'optimizer': ['adam', 'rmsprop']}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)

# Output the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

ValueError: 
All the 6 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "D:\Users\priyadarsani\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Users\priyadarsani\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 1465, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "D:\Users\priyadarsani\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 735, in fit
    self._fit(
  File "D:\Users\priyadarsani\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 887, in _fit
    X, y = self._initialize(X, y)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Users\priyadarsan

In [15]:
# MODEL WITH BEST HYPER PARAMETERS


best_model = Sequential()
#  hidden layers
best_model.add(Dense(256, input_dim=df_scaled.shape[1], activation='relu'))
best_model.add(Dense(128, activation='relu'))
# Output layer
best_model.add(Dense(len(y.unique()), activation='softmax'))


In [16]:
# Compile the best_model
best_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
best_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 256)                 │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 26)                  │           3,354 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40,602 (158.60 KB)

 Trainable params: 40,602 (158.60 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Verify the shapes are consistent
assert x_train.shape[0] == y_train.shape[0]
assert x_test.shape[0] == y_test.shape[0]

# Train the model
best_model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test,y_test))

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5394 - loss: 1.7334 - val_accuracy: 0.8210 - val_loss: 0.6144
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8412 - loss: 0.5428 - val_accuracy: 0.8735 - val_loss: 0.4214
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8841 - loss: 0.3887 - val_accuracy: 0.8995 - val_loss: 0.3304
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9107 - loss: 0.2895 - val_accuracy: 0.9302 - val_loss: 0.2460
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9352 - loss: 0.2220 - val_accuracy: 0.9333 - val_loss: 0.2310
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9423 - loss: 0.1897 - val_accuracy: 0.9337 - val_loss: 0.2130
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9524 - loss: 0.1564 - val_accuracy: 0.9435 - val_loss: 0.1792
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9578 - loss: 0.1344 - val_accuracy: 0.

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Step 1: Make Predictions
y_pred = best_model.predict(x_test)

# If using softmax activation in the output layer, convert probabilities to class labels
y_pred_classes = y_pred.argmax(axis=1)

# Step 2: Calculate Metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

# Step 3: Output the Metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step
Accuracy: 0.9667
Precision: 0.9676
Recall: 0.9667
F1 Score: 0.9668
